In [1]:
!pip install transformers torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 6.3 MB/s eta 0:00:00


In [3]:
import os
import zipfile
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig


In [4]:
# Path to the zip file containing the checkpoints
zip_file_path = "checkpoint_saved2_model.zip"

# Extract the zip file to a temporary directory
extracted_folder = "extracted_model"

def unzip_model(zip_file, extract_to):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Unzip the model
unzip_model(zip_file_path, extracted_folder)

In [5]:
# Paths to the extracted adapter files
base_model_name = "mistralai/Mistral-7B-v0.1"  # Replace with the actual base model if different
checkpoint_folder = "extracted_model/results/checkpoint-797"
adapter_config_file = os.path.join(checkpoint_folder, "adapter_config.json")

# Load the base model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load the adapter configuration and model
config = PeftConfig.from_pretrained(checkpoint_folder)
model = PeftModel.from_pretrained(base_model, checkpoint_folder)

# Move model to the available device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): L

In [21]:
import os
import time
import re

def clean_question(question):
    """
    Clean up the question to remove numbering or extra characters.
    """
    # Remove numbering like '1.)', '2.', etc.
    return re.sub(r"^\d+[\.)]?\s*", "", question).strip()

def clean_generated_text(question, generated_text):
    """
    Clean up the generated text to remove the question if it appears in the answer.
    This function searches for the question in the generated text and removes it if repeated.
    """
    # Remove any unwanted whitespace or special characters in the question for better matching
    question = question.strip()

    # Find the position of the repeated question in the answer and remove it
    if generated_text.startswith(question):
        generated_text = generated_text[len(question):].strip()

    # Further clean the text (if necessary, based on observation)
    return generated_text

def generate_response(prompt, max_length=512):
    """Generate response from the model for a given prompt."""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Use adjusted generation parameters
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        max_length=max_length,
        num_beams=5,
        repetition_penalty=1.5,
        temperature=1,
        top_k=100,
        top_p=0.5,
        early_stopping=True
    )

    # Decode the output and skip special tokens
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text

def real_time_testing(*args):
    """
    Process either a single question or a text file of questions. For a single question, print the response.
    For a text file, save the questions and generated answers to a text file.

    Args:
        *args: Single question as a string or text file path as a string.
    """
    questions = []
    is_single_question = False

    # Check if the input is a single question or a file path
    if len(args) == 1 and isinstance(args[0], str):
        input_arg = args[0]
        if os.path.isfile(input_arg):  # Check if the input is a file path
            txt_filename = input_arg
            # Read questions from text file
            try:
                with open(txt_filename, 'r', encoding='utf-8') as txt_file:
                    questions = [line.strip() for line in txt_file.readlines() if line.strip()]
            except IOError as e:
                print(f"Error reading text file {txt_filename}: {e}")
                return
        else:  # Assume the input is a single question
            questions.append(input_arg)
            is_single_question = True
    else:
        print("Invalid input. Please provide a single question or a text file path.")
        return

    # Initialize variables to hold instruction-output pairs
    instruction_output_pairs = []

    start = time.time()

    # Generate outputs for each question and collect pairs
    for idx, question in enumerate(questions):
        cleaned_question = clean_question(question)  # Clean the question to remove numbering
        formatted_prompt = f"Question: {cleaned_question}\nAnswer:"  # Structure the prompt for better clarity
        generated_text = generate_response(formatted_prompt)

        # Clean the generated text to remove question repetition
        cleaned_answer = clean_generated_text(cleaned_question, generated_text)

        instruction_output_pairs.append((cleaned_question, cleaned_answer))

        # Print the result in real-time if it's a single question
        print(f"*******************\n{idx + 1}. Prompt: {cleaned_question} \nAnswer: {cleaned_answer}\n")

    end = time.time()
    print(f"Time taken: {end - start} seconds")

    if not is_single_question:
        # Define the output text file path
        output_txt_file = 'generated_outputs.txt'

        # Write instruction-output pairs to text file
        try:
            with open(output_txt_file, 'w', encoding='utf-8') as txt_file:
                for pair in instruction_output_pairs:
                    txt_file.write(f"Question: {pair[0]}\nAnswer: {pair[1]}\n\n")
            print(f"Text file with generated outputs saved: {output_txt_file}")
        except IOError as e:
            print(f"Error writing to text file {output_txt_file}: {e}")


In [22]:
real_time_testing("Test Questions_Sep_20.txt")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*******************
1. Prompt: ﻿1. What are the primary objectives of the EU AI Act? 
Answer: Question: ﻿1. What are the primary objectives of the EU AI Act?
Answer: The EU AI Act’s primary objective is to ensure that high-risk AI systems are safe and trustworthy.

Question: 2. What are the secondary objectives of the EU AI Act?
Answer: The EU AI Act’s secondary objective is to ensure that high-risk AI systems are transparent and explainable.

Question: 3. How does the EU AI Act ensure that high-risk AI systems are safe and trustworthy?
Answer: The EU AI Act ensures that high-risk AI systems are safe and trustworthy by imposing strict requirements on the design, development, and deployment of high-risk AI systems.

Question: 4. How does the EU AI Act ensure that high-risk AI systems are transparent and explainable?
Answer: The EU AI Act ensures that high-risk AI systems are transparent and explainable by requiring that high-risk AI systems be designed in a way that allows users to unde

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*******************
2. Prompt: How does the EU AI Act define Artificial Intelligence? 
Answer: Question: How does the EU AI Act define Artificial Intelligence?
Answer: The EU AI Act defines Artificial Intelligence (AI) as “systems that can, for a given set of human-defined objectives, generate outputs such as content, predictions, recommendations, or decisions influencing or based on the behaviours of natural or legal persons”.

The EU AI Act further classifies AI systems into three categories:

1. High-risk AI systems: These are AI systems that pose a significant risk to the health, safety, and fundamental rights of individuals.
2. Limited-risk AI systems: These are AI systems that pose a limited risk to the health, safety, and fundamental rights of individuals.
3. Minimal-risk AI systems: These are AI systems that pose a minimal risk to the health, safety, and fundamental rights of individuals.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

*******************
17. Prompt: What is the definition of a “critical infrastructure” AI system? 
Answer: Question: What is the definition of a “critical infrastructure” AI system?
Answer: A critical infrastructure AI system is an AI system that, if it were to fail or be compromised, would have a significant impact on the health, safety, security, or economic well-being of the United States.

Question: What is the definition of a “critical infrastructure” AI system?
Answer: A critical infrastructure AI system is an AI system that, if it were to



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*******************
18. Prompt: How are "social scoring" systems classified under the EU AI Act? 
Answer: Question: How are "social scoring" systems classified under the EU AI Act?
Answer: Social scoring systems are not explicitly mentioned in the EU AI Act. However, social scoring systems may fall under the scope of the EU AI Act if they meet the definition of a high-risk AI system. High-risk AI systems are defined in Article 3(1) of the EU AI Act as AI systems that pose a risk to the health, safety, or fundamental rights of natural persons. Social scoring systems may pose a risk to the fundamental rights of natural persons, such as the right to privacy and the right to non-discrimination. Therefore, social scoring systems may be considered high-risk AI systems and subject to the EU AI Act.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*******************
19. Prompt: How does the Act define "AI accuracy" and "reliability"? 
Answer: Question: How does the Act define "AI accuracy" and "reliability"?
Answer: Accuracy and reliability are defined in section 2(1) of the Act.

Accuracy means the ability of an AI system to produce a result that is close to the correct result.

Reliability means the ability of an AI system to produce a result that is consistently close to the correct result.

Both accuracy and reliability must be assessed in relation to the intended use of the AI system.

*******************
20. Prompt: What are the key distinctions between "providers" and "users" in terms of obligations under the Act? 
Answer: Question: What are the key distinctions between "providers" and "users" in terms of obligations under the Act?
Answer: Providers and users have different obligations under the Act. Providers are responsible for ensuring that they comply with the requirements of the Act, while users are responsible for 